# Causal Inference 

> Correlation is not equal to causation

You may have seen this phrase before. It usually refers to the fact that just because we observe trends in data does not necessarily mean that one variable in the data *causes* changes in the other. There are many humorous examples of this, and the following provides many such examples:

https://www.tylervigen.com/spurious-correlations

However, we are often interested in answering causal questions, especially when it comes to health. Does smoking cause cancer? Does exercise decrease the risk of heart disease? Questions such as these all require causal analysis to answer. 

Causal inference is the branch of statistics which deals with these questions. The gold standard tool of causal inference is *randomization*, which is typically employed in randomized control trials (RCTs). In RCTs, subjects are randomized into 2 groups, one which receives an intervention and one which does not. Then, whatever effect is observed can be attributed to the intervention. 

However, what happens if we do not have a randomized control trial? How can we still make inferences about causal effects? We will look at this question in detail.

### Example 

Let's say we have built a model which identifies patients who are likely to benefit from palliative care. Whenever such a patient is identified in the hospital, the doctor is notified by email that their patient may benefit. We want to know whether this notification has a causal effect on certain outcomes, such as the percentage of patients who recieve palliative care services.  

Although this seems like a location where a randomized control trial would be perfect, randomized control trials are 

    1. Expensive
    2. Time-consuming
    3. Ethically complicated
    


If we can somehow mimic the results of a randomized control trial, while using observational data, that would be a good start towards showing that this intervention had a real effect. 

## Rubin Causal Model 

Donald Rubin, in his seminal papers in '74 and '78, defined a new framework for doing causal inference. There is also work done by Judea Pearl, which is another popular framework, but which we will not discuss in this class. In his model, Rubin relies on a concept known as the *Potential Outcome*.

#### Potential Outcome

A **potential outcome** is the value of a unit's measurement of interest after treatment or non-treatment. However, the fundamental problem in causal inference is that we can only observe one of these measurements! Either we get the value after treatment or if a unit does not receive the treatment. So essentially this boils down to a missing data problem 

## Assumptions of RCM

Before we can just go ahead and compute the causal effect of some intervention, we have to make sure that we meet certain assumptions. These are very important in the RCM since if any of these are violated, the entire analysis could be invalid. The most important assumption is the SUTVA assumption, which stands for Stable Unit Treatment Value Assumption. 

## SUTVA 

This assumption states that the potential outcomes for any unit do not vary with the treatments assigned to any other units AND that there are no different versions of the treatment. 

In some studies, it is very easy to violate these assumptions. For example, if you are studying the effect of some contagious disease, it is clear that the the potential outcome for one unit could vary with another since it may make them more/less likely to spread the disease. There is still ongoing research to determine whether other assumptions are required if SUTVA is violated.

In addition, if we are working with observational data, we need a few more assumptions. In particular, we do not control or know 